### Import

In [1]:
import os
import itertools
import time
import pandas as pd
import numpy as np

from dfg_rating.model import factory
import dfg_rating.viz.jupyter_widgets as DFGViz

from dfg_rating.model.rating.elo_rating import ELORating
from dfg_rating.model.network.base_network import WhiteNetwork
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction


from dfg_rating.model.betting.betting import FixedBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore, Likelihood, ProbabilityDifference, ProbabilityPointer, FavouriteProbability
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.evaluators.base_evaluators import BettingActivity

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Loading real data

In [2]:
data_football_national = pd.read_csv(os.path.join('..', '..', '..', 'data', 'real', 'Data_Football_National.csv'),sep = ";")
#data_football_national = pd.read_csv("C:/Users/Dell-PC/Documents/Projekte/dfg-rating/data/real/Data_Football_National.csv")

New mapping available:
 - ts_format: Timestamp format
 - winner with result and translation to (home, draw, away).

In [3]:
football_network = WhiteNetwork(
    data=data_football_national,
    #node1 = away
    mapping={
        "node1": {
            "id": "AwayID",
            "name": "AwayTeam",
        },
        "node2": {
            "id": "HomeID",
            "name": "HomeTeam",
        },
        "day": "Date",
        "dayIsTimestamp": True,
        "ts_format": "%d.%m.%Y",
        "tournament": "Div",
        "season": "Season",
        "winner": {
            "result": "ResultFT",
            "translation": {
                "H": "home",
                "D": "draw",
                "A": "away"
            }
        },
        "round": "day",
        "odds": {
            "maximumodds": {
                "home": "OddsHomeMax",
                "draw": "OddsDrawMax",
                "away": "OddsAwayMax"
            },
            "averageodds": {
                "home": "OddsHomeAvg",
                "draw": "OddsDrawAvg",
                "away": "OddsAwayAvg"
            },
        },
        "bets": {}
    }
)

Network loaded correctly


### Adding ratings to the loaded network

In [4]:
football_network.add_rating(
    rating=ELORating(
        trained=True, 
        rating_name='player_rating',
        **{
            'param_c' : 10,
            'param_d' : 400,
            'param_k' : 50,
            'param_w' : 50
        }
    ),
    rating_name='player_rating'
)

Rating explorer (Only useful the first chart, ignore the others)

In [5]:
app = DFGViz.RatingsExplorer(
    network=football_network
)

In [6]:
app.run('inline')

### Adding a forecast from the added rating

In [7]:
football_network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.2, 0.0], beta_parameter=0.006),
    forecast_name='player_forecast',
    base_ranking='player_rating'
)
football_network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-0.9, 0.3], beta_parameter=0.006),
    forecast_name='true_forecast',
    base_ranking='player_rating'
)

Exploring the forecasts manually

In [8]:
football_network.get_seasons()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
app_f = DFGViz.ForecastExplorer(
    network=football_network,
    ratings=["player_rating"]
)

In [10]:
app_f.run('inline')

### Inside loop functions

In [11]:
def add_elo_rating(k, n, name):
    n.add_rating(
    rating=ELORating(
        trained=True, 
        rating_name=name,
        **{
            'param_c' : 10,
            'param_d' : 400,
            'param_k' : k,
            'param_w' : 50
        }
    ),
    rating_name=name
)

In [12]:
def aggregate_measures(list_of_matches, prefix='', addBettingMetrics=True):
    if addBettingMetrics:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches]),
            prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
            prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches]),
            prefix+'_betting_activity': sum([m[3]['metrics']['betting_activity']['qty'] for m in list_of_matches])
        }
    else:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches])
        }

## Betting config

We create a bookmaker and a betting strategy that are going to interact with the generated networks.

In [16]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)

In [17]:
betting = FixedBetting(100)

We also create an error rating that we are going to use for the calculated forecasts

In [18]:
rating_error = RatingFunctionError(error='normal', loc=config["rating_error_loc"], scale=config["rating_error_scale"])
rating_error_bookmaker = RatingFunctionError(error='normal', loc=config["bookmaker_rating_error_loc"], scale=config["bookmaker_rating_error_scale"])

NameError: name 'RatingFunctionError' is not defined

### Small loops to test

Small options

In [19]:
k_options = range(15,16,1)
c0_options = np.arange(-1.20, -1.00, 0.1)
c1_options = np.arange(0.0, 0.2, 0.1)
beta_options = np.arange(0.006, 0.008, 0.002)

Full options

In [20]:
k_options = range(20,30,10)
c0_options = np.arange(-1.20, -0.60, 0.1)
c1_options = np.arange(0.0, 0.60, 0.1)
beta_options = np.arange(0.002, 0.012, 0.002)

In [21]:
experiment_start_time = time.time()
result_list = []
for k in k_options:
    rating_name = f"elo_rating_{k}"
    print(f"Rating <{rating_name}>")
    add_elo_rating(k, football_network, rating_name)
    for c0, c1, beta in itertools.product(c0_options, c1_options, beta_options):
        print(f'Variables: c0: {c0}, c1: {c1} and beta: {beta}')
        forecast_pointer = f"player_{k}_forecast_{c0:.2f}_{c1:.2f}_{beta:.3f}"
        cell_start_time = time.time()
        football_network.add_forecast(
            forecast=LogFunctionForecast(
                outcomes=['home', 'draw', 'away'], 
                coefficients=[c0, c1], 
                beta_parameter=beta
            ),
            forecast_name=forecast_pointer,
            base_ranking=rating_name
        )
        football_network.add_bets(
            bettor_name='b',
            bookmaker='averageodds',
            betting=betting,
            base_forecast=forecast_pointer
        )
        rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)
        betting_returns = BettingReturnsEvaluator(
            outcomes=['home', 'draw', 'away'], 
            player_name='b', 
            true_model=forecast_pointer, 
            bookmaker_name='averageodds'
        )
        betting_activity = BettingActivity(outcomes=['home', 'draw', 'away'], player_name='b')
        likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)
        difference = ProbabilityDifference(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)
        draw_probability = ProbabilityPointer(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer, probability_index=1)
        fav_probability = FavouriteProbability(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)

        football_network.add_evaluation([
            (rps, 'rps'),
            (betting_returns, 'betting_returns'),
            (betting_activity, 'betting_activity'),
            (likelihood, 'likelihood'),
            (difference, 'prob_diff'),
            (draw_probability, 'draw_prob'),
            (fav_probability, 'fav_prob')
        ])
        all_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in football_network.iterate_over_games()]
        result_all = aggregate_measures(all_matches, 'all')
        result_is = aggregate_measures([m for m in all_matches if m[3]['season'] <= 3], 'is')
        result_oos = aggregate_measures([m for m in all_matches if m[3]['season'] > 3], 'oos')
        result = {
            'k': k, 'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all
        }
        result_list.append(result)
        print(f"Finished in {float(time.time() - cell_start_time)} seconds")

print(f"Experiment finished in {float(time.time() - experiment_start_time)} seconds with {len(result_list)} observations.")
df = pd.DataFrame(result_list)

Rating <elo_rating_20>
Variables: c0: -1.2, c1: 0.0 and beta: 0.002
Finished in 6.793362140655518 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.004
Finished in 6.547930955886841 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.006
Finished in 6.881473779678345 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.008
Finished in 6.757410764694214 seconds
Variables: c0: -1.2, c1: 0.0 and beta: 0.01
Finished in 6.1215455532073975 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.002
Finished in 6.432885408401489 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.004
Finished in 6.50680136680603 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.006
Finished in 6.583940744400024 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.008
Finished in 6.119697332382202 seconds
Variables: c0: -1.2, c1: 0.1 and beta: 0.01
Finished in 6.669501066207886 seconds
Variables: c0: -1.2, c1: 0.2 and beta: 0.002
Finished in 6.105313777923584 seconds
Variables: c0: -1.2, c1: 0.2 and beta: 0.004
Finished in 6.6790220

In [22]:
df

,k,c0,c1,beta,is_rps,is_likelihood,is_prob_diff,is_draw_prob,is_fav_prob,is_betting_returns,is_betting_returns_expected,is_betting_activity,oos_rps,oos_likelihood,oos_prob_diff,oos_draw_prob,oos_fav_prob,oos_betting_returns,oos_betting_returns_expected,oos_betting_activity,all_rps,all_likelihood,all_prob_diff,all_draw_prob,all_fav_prob,all_betting_returns,all_betting_returns_expected,all_betting_activity
0,20,-1.20000,0.00000,0.00200,0.21725,-15541.21513,0.26661,0.26716,0.49979,-1801.59000,5666.30869,15188,0.21678,-36384.02007,0.26535,0.26576,0.50061,-4463.58000,14602.21654,38406,0.21692,-51925.23520,0.26573,0.26618,0.50037,-6265.17000,20268.52523,53594
1,20,-1.20000,0.00000,0.00400,0.21267,-15336.96653,0.26203,0.26311,0.50393,-1796.97000,4022.83267,14463,0.21056,-35725.85670,0.25729,0.25809,0.51350,-3753.17000,8693.59834,35716,0.21119,-51062.82324,0.25871,0.25959,0.51064,-5550.14000,12716.43101,50179
2,20,-1.20000,0.00000,0.00600,0.21072,-15250.03284,0.25536,0.25694,0.51535,-1582.63000,2975.41281,13430,0.20906,-35578.45765,0.24611,0.24730,0.53564,-3002.42000,5879.71244,32428,0.20956,-50828.49048,0.24887,0.25018,0.52958,-4585.05000,8855.12525,45858
3,20,-1.20000,0.00000,0.00800,0.21099,-15271.90334,0.24729,0.24932,0.53131,-1347.12000,2625.90237,12913,0.21100,-35874.43338,0.23360,0.23515,0.56142,-2415.10000,5960.85863,31884,0.21100,-51146.33672,0.23769,0.23938,0.55243,-3762.22000,8586.76100,44797
4,20,-1.20000,0.00000,0.01000,0.21297,-15391.62916,0.23844,0.24088,0.54960,-1355.05000,2767.38294,13097,0.21513,-36539.02637,0.22092,0.22281,0.58746,-2387.49000,7287.94698,32393,0.21449,-51930.65553,0.22615,0.22821,0.57615,-3742.54000,10055.32992,45490
5,20,-1.20000,0.10000,0.00200,0.21653,-15526.25526,0.24182,0.29195,0.47509,-2031.51000,5430.22385,16360,0.21575,-36320.39481,0.24074,0.29037,0.47646,-4794.93000,14217.67837,43038,0.21599,-51846.65007,0.24106,0.29085,0.47605,-6826.44000,19647.90222,59398
6,20,-1.20000,0.10000,0.00400,0.21196,-15315.77581,0.23775,0.28739,0.48070,-2069.24000,3760.22078,15602,0.20954,-35637.89549,0.23361,0.28177,0.49176,-4236.21000,8149.58085,40096,0.21026,-50953.67130,0.23485,0.28345,0.48846,-6305.45000,11909.80163,55698
7,20,-1.20000,0.10000,0.00600,0.21001,-15223.15070,0.23183,0.28047,0.49407,-1744.46000,2685.48432,14501,0.20806,-35470.01373,0.22366,0.26974,0.51647,-3210.10000,5205.95031,35942,0.20864,-50693.16442,0.22610,0.27295,0.50978,-4954.56000,7891.43463,50443
8,20,-1.20000,0.10000,0.00800,0.21029,-15239.99521,0.22463,0.27198,0.51187,-1446.74000,2339.33096,14010,0.21001,-35749.46493,0.21245,0.25629,0.54435,-2719.82000,5337.52619,35005,0.21009,-50989.46014,0.21609,0.26098,0.53465,-4166.56000,7676.85715,49015
9,20,-1.20000,0.10000,0.01000,0.21228,-15355.35966,0.21671,0.26261,0.53176,-1452.30000,2497.39302,14164,0.21416,-36400.87925,0.20105,0.24267,0.57206,-2506.31000,6719.70024,35465,0.21360,-51756.23891,0.20573,0.24863,0.56002,-3958.61000,9217.09325,49629


In [23]:
df.to_excel(f"Results_RealWorldAVGMoreSpecs.xlsx")